In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import asyncio
import logging

from app.redis_transcribe.connection import get_redis_client
from app.settings import settings
from app.services.transcription.processor import Processor




redis_client = await get_redis_client(settings.redis_host, settings.redis_port,settings.redis_password)

processor = Processor(redis_client,max_length=30)



2025-02-18 12:38:12,345 - INFO - Attempting Redis connection to redis:6379
2025-02-18 12:38:12,346 - INFO - Redis password is configured
2025-02-18 12:38:12,349 - INFO - Successfully connected to Redis server


In [4]:
from app.services.audio.redis_models import Transcript

In [5]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

In [6]:
from app.services.audio.redis_models import Connection, Meeting, Transcriber

In [126]:
transcriber = Transcriber(redis_client)
await transcriber.add_todo('the-zdjv-byg')
df = await Transcript.get_all_transcripts_df(redis_client)
df.drop(columns=['words']).tail(20)

,content,start_timestamp,end_timestamp,speaker,confidence,segment_id,meeting_id,server_timestamp,transcription_timestamp
17,и внутри компании все это разворачиваешь.,2025-02-13 15:45:19.676000+00:00,2025-02-13 15:45:21.896000+00:00,Dmitriy Grankin,1.000000,187,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
18,"Ты разворачивай, пожалуйста, пользуйся.",2025-02-13 15:45:22.156000+00:00,2025-02-13 15:45:27.336000+00:00,Dmitriy Grankin,0.355985,188,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
19,"Если ты хочешь взять это просто опубликовать как VEX.2 и поставить цену не 10 долларов, а 5 долларов, решение.",2025-02-13 15:45:27.876000+00:00,2025-02-13 15:45:38.696000+00:00,Dmitriy Grankin,0.896118,189,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
20,"Ага. Ну, да, логично, логично.",2025-02-13 15:45:39.276000+00:00,2025-02-13 15:45:44.706000+00:00,Dmitriy Grankin,0.869982,190,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
21,"Вот, ну, слушай, отлично.",2025-02-13 15:45:48.556000+00:00,2025-02-13 15:45:51.316000+00:00,Dmitriy Grankin,0.839130,191,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
22,Я как бы...,2025-02-13 15:45:51.556000+00:00,2025-02-13 15:45:52.376000+00:00,Dmitriy Grankin,1.000000,192,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
23,"Окей, ладно.",2025-02-13 15:45:55.596000+00:00,2025-02-13 15:45:56.256000+00:00,Dmitriy Grankin,0.232727,193,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
24,"Так, ладненько, я тут заехал, да, заехал уже на парковку, может, звук.",2025-02-13 15:45:56.476000+00:00,2025-02-13 15:46:00.936000+00:00,Dmitriy Grankin,0.205022,194,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
25,"Давай, пока, пока.",2025-02-13 15:46:01.616000+00:00,2025-02-13 15:46:02.916000+00:00,Sergey Ryabenko,1.000000,195,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:16.421187+00:00
10,Пока-пока.,2025-02-13 15:46:03.328000+00:00,2025-02-13 15:46:03.808000+00:00,Sergey Ryabenko,1.000000,196,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 12:44:17.260962+00:00


In [74]:
# Add debug logging before getting transcripts
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Get and inspect raw data first
raw_transcripts = await redis_client.keys('transcript:*')
logger.debug(f"Found transcript keys: {raw_transcripts}")

df = await Transcript.get_all_transcripts_df(redis_client)
logger.debug(f"DataFrame columns: {df.columns}")
logger.debug(f"DataFrame shape: {df.shape}")
df.tail(10)

,content,start_timestamp,end_timestamp,speaker,confidence,segment_id,meeting_id,words,server_timestamp,transcription_timestamp
2,"Ah, eu ponho, ok.",2025-02-13 15:32:16.430000+00:00,2025-02-13 15:32:19.190000+00:00,None,0.000000,1,the-zdjv-byg,"[[ Ah,, 0.43, 0.99], [ eu, 1.27, 1.57], [ ponho,, 1.57, 2.55], [ ok., 2.77, 3.19]]",2025-02-13 15:32:16+00:00,2025-02-18 12:33:28.077214+00:00
1,"Думал, ты что-то про, ну типа, как хранить там и так далее. Ну окей.",2025-02-13 15:32:21.040000+00:00,2025-02-13 15:32:25.600000+00:00,Sergey Ryabenko,1.000000,2,the-zdjv-byg,"[[ Думал,, 0.0, 0.24], [ ты, 0.24, 0.34], [ что, 0.34, 0.48], [-то, 0.48, 0.64], [ про,, 0.64, 0.84], [ ну, 1.04, 1.76], [ типа,, 1.76, 2.0], [ как, 2.02, 2.28], [ хранить, 2.28, 2.84], [ там, 2.84, 2.98], [ и, 2.98, 3.08], [ так, 3.08, 3.22], [ далее., 3.22, 3.54], [ Ну, 3.7, 4.1], [ окей., 4.1, 4.56]]",2025-02-13 15:32:16+00:00,2025-02-18 12:33:30.195662+00:00
0,"Да, нужно записать.",2025-02-13 15:32:26.260000+00:00,2025-02-13 15:32:30.760000+00:00,Sergey Ryabenko,0.088889,3,the-zdjv-byg,"[[ Да,, 0.18, 2.16], [ нужно, 2.32, 2.6], [ записать., 2.6, 4.68]]",2025-02-13 15:32:16+00:00,2025-02-18 12:33:32.266400+00:00


In [75]:
df

,content,start_timestamp,end_timestamp,speaker,confidence,segment_id,meeting_id,words,server_timestamp,transcription_timestamp
2,"Ah, eu ponho, ok.",2025-02-13 15:32:16.430000+00:00,2025-02-13 15:32:19.190000+00:00,None,0.000000,1,the-zdjv-byg,"[[ Ah,, 0.43, 0.99], [ eu, 1.27, 1.57], [ ponho,, 1.57, 2.55], [ ok., 2.77, 3.19]]",2025-02-13 15:32:16+00:00,2025-02-18 12:33:28.077214+00:00
1,"Думал, ты что-то про, ну типа, как хранить там и так далее. Ну окей.",2025-02-13 15:32:21.040000+00:00,2025-02-13 15:32:25.600000+00:00,Sergey Ryabenko,1.000000,2,the-zdjv-byg,"[[ Думал,, 0.0, 0.24], [ ты, 0.24, 0.34], [ что, 0.34, 0.48], [-то, 0.48, 0.64], [ про,, 0.64, 0.84], [ ну, 1.04, 1.76], [ типа,, 1.76, 2.0], [ как, 2.02, 2.28], [ хранить, 2.28, 2.84], [ там, 2.84, 2.98], [ и, 2.98, 3.08], [ так, 3.08, 3.22], [ далее., 3.22, 3.54], [ Ну, 3.7, 4.1], [ окей., 4.1, 4.56]]",2025-02-13 15:32:16+00:00,2025-02-18 12:33:30.195662+00:00
0,"Да, нужно записать.",2025-02-13 15:32:26.260000+00:00,2025-02-13 15:32:30.760000+00:00,Sergey Ryabenko,0.088889,3,the-zdjv-byg,"[[ Да,, 0.18, 2.16], [ нужно, 2.32, 2.6], [ записать., 2.6, 4.68]]",2025-02-13 15:32:16+00:00,2025-02-18 12:33:32.266400+00:00
